# Proyecto B Etapa 2 Modelado, Simulación y Optimización

## Caso 1

In [17]:
import pandas as pd
import requests
import time

In [20]:
import pandas as pd
import requests
import time

# 1. Carga de datos
clients = pd.read_csv("data-caso1/clients.csv")
depots  = pd.read_csv("data-caso1/depots.csv")

# 2. Construcción de nodos con coordenadas
nodes = {0: (depots.loc[0, "Longitude"], depots.loc[0, "Latitude"])}
for _, row in clients.iterrows():
    nodes[int(row["ClientID"])] = (row["Longitude"], row["Latitude"])

# 3. Configuración de la API
api_key = '5b3ce3597851110001cf6248f4bace05742cd52a0f6b877ce5d52bffe229f05a5f3132ac87cef6d3'
headers = {
    'Authorization': api_key,
    'Content-Type': 'application/json'
}
url = 'https://api.openrouteservice.org/v2/directions/driving-car'

# 4. Creación de grafo con tuplas
grafo1 = {}

for i, coords_i in nodes.items():
    grafo1[i] = {}
    for j, coords_j in nodes.items():
        if i == j:
            # distancia y tiempo "infinitos" para mismo nodo
            grafo1[i][j] = (999.0, 999.0)
            continue

        params = {
            'start': f"{coords_i[0]},{coords_i[1]}",
            'end':   f"{coords_j[0]},{coords_j[1]}"
        }

        resp = requests.get(url, headers=headers, params=params)
        time.sleep(1.1)  # para no pasarte del límite

        if resp.status_code == 200:
            seg = resp.json()['features'][0]['properties']['segments'][0]
            dist = round(seg['distance'] / 1000, 2)  # km
            tiem = round(seg['duration'] / 60,   2)  # minutos

            # convertir ceros a 999.0
            if dist == 0.0: dist = 999.0
            if tiem == 0.0: tiem = 999.0

            grafo1[i][j] = (dist, tiem)
        else:
            # en caso de error HTTP
            grafo1[i][j] = (999.0, 999.0)
            print(f"Error {resp.status_code} entre {i} y {j}")

# 5. Mostrar resultado
import pprint
pprint.pprint(grafo1, width=60)



KeyboardInterrupt: 

In [ ]:
import json

# Cargar el archivo JSON
with open('distancias-tiempo-api.json', 'r') as f:
    data = json.load(f)

# Crear la variable grafo
grafo1 = {}

for origen, destinos in data.items():
    grafo1[int(origen)] = {}
    for destino, valores in destinos.items():
        distancia = valores['distance_km']
        tiempo = valores['time_min']
        if distancia == 0.0:
            distancia = 999.0
        if tiempo == 0.0:
            tiempo = 999.0
        grafo1[int(origen)][int(destino)] = (distancia, tiempo)




In [21]:
print(grafo1)

{0: {0: (999.0, 999.0), 1: (27.19, 37.94)}}


In [ ]:
from pyomo.environ import (
    ConcreteModel, Set, Param, Var, Binary, NonNegativeReals,
    Objective, Constraint, minimize, SolverFactory
)
import pandas as pd

# Datos 
clients = pd.read_csv("data-caso1/clients.csv")
vehicles = pd.read_csv("data-caso1/vehicles.csv")

clientes = clients["ClientID"].tolist()
vehiculos = vehicles["VehicleID"].tolist()
nodos = [0] + clientes

demanda = dict(zip(clients["ClientID"], clients["Demand"]))
capacidad = dict(zip(vehicles["VehicleID"], vehicles["Capacity"]))


distancias = {(i,j): grafo1[i][j][0] for i in nodos for j in nodos if i!=j}
tiempos    = {(i,j): grafo1[i][j][1] for i in nodos for j in nodos if i!=j}

# Construcción del modelo 
model = ConcreteModel()

model.N = Set(initialize=nodos)
model.C = Set(initialize=clientes)
model.V = Set(initialize=vehiculos)

model.d = Param(model.C, initialize=demanda) # demanda de los clientes
model.q = Param(model.V, initialize=capacidad) # capacidad de los vehículos
model.dist = Param(model.N, model.N, initialize=distancias, default=1e6)
model.t = Param(model.N, model.N, initialize=tiempos, default=0)

# Costos 
model.Pf = Param(initialize=15000)   # precio combustible por litro
model.Cm = Param(initialize=700)     # costo mantenimiento por km
model.Ce = Param(initialize=0)       # costo eléctrico
model.Ft = Param(initialize=5000)    # tarifa fija por km
model.Cv = Param(initialize=0)       # costo por viaje

# δ_v = 1 para todos los vehículos (todos camionetas)
delta = { v: 1 for v in vehiculos }
model.delta = Param(model.V, initialize=delta)

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)
model.u = Var(model.C, model.V, domain=NonNegativeReals)

# Función Objetivo
def obj_custom(m):
    return sum((m.Pf * m.dist[i,j] * m.delta[v] + m.Cm + m.delta[v] * m.Ce + m.Ft + m.Cv * m.t[i,j]) * m.x[i,j,v] for i in m.N for j in m.N for v in m.V if i!=j)
model.obj = Objective(rule=obj_custom, sense=minimize)



# Restricciones
def cliente_una_vez(m, j):
    return sum(m.x[i,j,v] for i in m.N for v in m.V if i!=j) == 1

def cliente_salida(m, j):
    return sum(m.x[j,k,v] for k in m.N for v in m.V if j!=k) == 1

def flujo_cliente(m, j, v):
    return (sum(m.x[i,j,v] for i in m.N if i!=j) - sum(m.x[j,k,v] for k in m.N if k!=j)) == 0

def mtz_rule(m, i, j, v):
    if i!=j:
        return m.u[i,v] + m.d[j] - m.u[j,v] <= m.q[v]*(1 - m.x[i,j,v])
    return Constraint.Skip

def limites_u(m, i, v):
    return (m.d[i], m.u[i,v], m.q[v])

def capacidad_total(m, v):
    return sum(m.d[j]*m.x[i,j,v]
               for i in m.N for j in m.C if i!=j) <= m.q[v]


model.cliente_una_vez = Constraint(model.C, rule=cliente_una_vez)
model.cliente_salida = Constraint(model.C, rule=cliente_salida)
model.flujo = Constraint(model.C, model.V, rule=flujo_cliente)
model.salida_dep = Constraint(model.V, rule=lambda m,v: sum(m.x[0,j,v] for j in m.C) <= 1)
model.entrada_dep = Constraint(model.V, rule=lambda m,v: sum(m.x[i,0,v] for i in m.C) <= 1)
model.mtz = Constraint(model.C, model.C, model.V, rule=mtz_rule)
model.lim_u = Constraint(model.C, model.V, rule=limites_u)
model.cap_total = Constraint(model.V, rule=capacidad_total)

# Solución del modelo
solver = SolverFactory("glpk")
solver.options["tmlim"]  = 60     # segundos máximo
solver.options["mipgap"] = 0.05   # gap 5%
results = solver.solve(model, tee=True)

print("Status:     ", results.solver.status)
print("Termination:", results.solver.termination_condition)
print("Obj. value: ", model.obj())

# Print resultados
print("\n RUTAS ASIGNADAS POR VEHÍCULO:")
for v in model.V:
    rutas, actual, visitados = [], 0, set()
    while True:
        nxt = next(
            (j for j in model.N if j!=actual and model.x[actual,j,v].value>0.5),
            None
        )
        if nxt is None or nxt in visitados:
            break
        rutas.append((actual, nxt))
        visitados.add(nxt)
        actual = nxt
    print(f" Vehículo {v}: {rutas or 'no asignado'}")




GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 60 --mipgap 0.05 --write C:\Users\57314\AppData\Local\Temp\tmp5z1ptty2.glpk.raw
 --wglp C:\Users\57314\AppData\Local\Temp\tmpzbudnfzu.glpk.glp --cpxlp C:\Users\57314\AppData\Local\Temp\tmp0d92w451.pyomo.lp
Reading problem data from 'C:\Users\57314\AppData\Local\Temp\tmp0d92w451.pyomo.lp'...
C:\Users\57314\AppData\Local\Temp\tmp0d92w451.pyomo.lp:62050: warning: lower bound of variable 'x2' redefined
C:\Users\57314\AppData\Local\Temp\tmp0d92w451.pyomo.lp:62050: warning: upper bound of variable 'x2' redefined
5064 rows, 4992 columns, 37056 non-zeros
4800 integer variables, all of which are binary
66850 lines were read
Writing problem data to 'C:\Users\57314\AppData\Local\Temp\tmpzbudnfzu.glpk.glp'...
56979 lines were written
GLPK Integer Optimizer 5.0
5064 rows, 4992 columns, 37056 non-zeros
4800 integer variables, all of which are binary
Preprocessing...
4680 rows, 4992 columns, 36672 non-zeros
4800 integ

## Caso 2

In [ ]:
import json
import pprint

# Cargar el archivo JSON 
with open('distancias-tiempo-api-carro-2.json', 'r') as f:
    data = json.load(f)

# Reconstruir grafo
grafo2 = {}
for origen_str, destinos in data.items():
    origen = int(origen_str)
    grafo2[origen] = {}
    for destino_str, valores in destinos.items():
        destino = int(destino_str)

        dist, tiem = valores

        if dist == 0.0: dist = 999.0
        if tiem == 0.0: tiem = 999.0
        grafo2[origen][destino] = (dist, tiem)




In [26]:
print(grafo2)

{0: {0: (999.0, 999.0), 1: (999.0, 999.0), 2: (999.0, 999.0), 3: (999.0, 999.0), 4: (999.0, 999.0), 5: (999.0, 999.0), 6: (999.0, 999.0), 7: (999.0, 999.0), 8: (999.0, 999.0), 9: (999.0, 999.0), 10: (999.0, 999.0), 11: (999.0, 999.0), 12: (999.0, 999.0), 13: (999.0, 999.0), 14: (999.0, 999.0)}, 1: {0: (999.0, 999.0), 1: (999.0, 999.0), 2: (14.73, 42.21), 3: (9.02, 24.12), 4: (4.3, 12.99), 5: (4.46, 13.49), 6: (999.0, 999.0), 7: (999.0, 999.0), 8: (11.43, 28.93), 9: (11.92, 33.63), 10: (1.58, 4.77), 11: (7.88, 22.33), 12: (999.0, 999.0), 13: (3.95, 11.94), 14: (999.0, 999.0)}, 2: {0: (999.0, 999.0), 1: (14.73, 42.21), 2: (999.0, 999.0), 3: (9.18, 25.02), 4: (10.43, 29.22), 5: (13.35, 37.97), 6: (999.0, 999.0), 7: (999.0, 999.0), 8: (11.59, 29.84), 9: (2.83, 8.63), 10: (16.1, 46.28), 11: (7.81, 22.77), 12: (999.0, 999.0), 13: (13.76, 39.33), 14: (999.0, 999.0)}, 3: {0: (999.0, 999.0), 1: (9.02, 24.12), 2: (9.18, 25.02), 3: (999.0, 999.0), 4: (4.72, 11.13), 5: (7.64, 19.88), 6: (999.0, 99

## Caso 3
